# This script merges the orders_prior and product datasets

### This script contains these sections

#### 1. Importing Libraries
#### 2. Importing Data
#### 3. Checking Dimensions of data
#### 4. Merging Products and Orders_Prior Datasets
#### 5. Exporting Merged Dataset

# 1. Importing Libraries

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os

# 2. Importing Data

In [2]:
# Creating shortcut to folder
path = r'C:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Data Immersion\4 Python\03-2020_Instacart_Basket _Analysis'

In [3]:
# Importing Products Data that was already wrangled and checked
df_prods = pd.read_csv(os.path.join(path, '02_Data', 'Prepared_Data', 'products_checked.csv'),
                                 index_col = False)

In [4]:
# Importing Orders_prior data that was already merged 
df_ords_prods_comb = pd.read_pickle(os.path.join(path, '02_Data', 'Prepared_Data', 'orders_products_combined.pkl'))

# 3. Checking Dimensions of Data

In [5]:
df_ords_prods_comb.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [6]:
# Check the shape of the imported dataframe 
df_ords_prods_comb.shape

(32434489, 10)

This is the same as when I exported it, good

In [7]:
# Now checking cols of products before merging
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [8]:
# Double-Checking the issue with price is resolved in imported df
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,7.680326
std,14340.705287,38.315784,5.850779,4.199349
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,25.000000


# 4. Merging products and orders data

In [9]:
# since old merged column already had a merge flag, dropping this before new merge
df_ords_prods_comb.drop(columns=['_merge'], 
                        inplace=True)

In [ ]:
# Using inner join since it was specified that we only want full data
    # Can use product_id since it has same name in both DFs
df_ords_prods = df_ords_prods_comb.merge(df_prods,
                                         on = 'product_id',
                                         how='inner',
                                         indicator=True)

In [11]:
df_ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,4.4,both
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


In [12]:
df_ords_prods.shape

(32404859, 14)

In [13]:
# Checking results
df_ords_prods['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [14]:
# Testing merge using outer to see if/how it differs
df_test = df_ords_prods_comb.merge(df_prods,
                                         on = 'product_id',
                                         how='outer',
                                         indicator=True)

In [15]:
df_test['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

We can see that there were 30200 Records from the orders df and 11 from product that were left out of merge

# 5. Exporting combined data as Pickle

In [16]:
# Exporting as pickle since it's a very large file and next steps will be performed with python
df_ords_prods.to_pickle(os.path.join(path, '02_Data', 'Prepared_Data', 'ords_prods_merge.pkl'))